In [22]:
import pandas as pd
import numpy as np
import copy

In [32]:
doc_name = "./data/Паспорт_21.02"
dataset = pd.read_excel(f"{doc_name}.xls")

In [33]:
dataset.iloc[2]

Паспорт семейно-демографических показателей Волгоградской области\nТаблица 1.  01.02.2022              NaN
Unnamed: 1                                                                                             NaN
Unnamed: 2                                                                                             NaN
Unnamed: 3                                                                                             NaN
Unnamed: 4                                                                                             NaN
                                                                                                 ...      
Unnamed: 79                                                                                            NaN
Unnamed: 80                                                                                  с 12-ю детьми
Unnamed: 81                                                                                  с 13-ю детьми
Unnamed: 82                          

In [34]:
columns_names = [
	"Наименование района",
	"Всего семей",
	"Всего детей",
	"Семьи с 2-мя детьми-близнецами",
	"Семьи с 3-мя детьми-близнецами",
]
object_part = {
	"column_pos": [],
	"data": []
}
columns_data = {col: copy.deepcopy(object_part) for col in columns_names}
columns_first_finded = {col: False for col in columns_names}
columns_amount = len(dataset.columns)
columns_finded = len(columns_names)
table_end_index_row = 0
search_table_start_index = 0

def search_table_start_by_name(dataset, search_phrase):
	index_phrase = 0
	for i in range(len(dataset)):
		row = dataset.iloc[i]
		row = map(str, list(row.values))
		str_row = "".join(row)

		if search_phrase in str_row:
			# print(row)
			index_phrase = i
			break
	print("index_phrase", index_phrase)
	return index_phrase

search_phrase = "Семьи с близнецами"
search_table_start_index = search_table_start_by_name(
	dataset=dataset,
	search_phrase=search_phrase
)

for i in range(search_table_start_index, len(dataset)):
	row = dataset.iloc[i]
	row = map(str, list(row.values))
	str_row = "".join(row)
	if columns_finded > 0:
		for col_name in columns_names:
			if col_name in str_row and not columns_first_finded[col_name]:
				for j in range(columns_amount):
					col_data = dataset.iloc[i, j]
					if col_name == col_data:
						columns_data[col_name]['column_pos'] = [i, j]
						columns_finded -= 1
						columns_first_finded[col_name] = True
						break

	if "области" in str_row:
		# print(str_row)
		table_end_index_row = i - 1
		break 

index_phrase 246


In [25]:
table_end_index_row

241

In [35]:
columns_data

{'Наименование района': {'column_pos': [248, 11], 'data': []},
 'Всего семей': {'column_pos': [248, 25], 'data': []},
 'Всего детей': {'column_pos': [248, 53], 'data': []},
 'Семьи с 2-мя детьми-близнецами': {'column_pos': [], 'data': []},
 'Семьи с 3-мя детьми-близнецами': {'column_pos': [], 'data': []}}

In [26]:
dataset.iloc[62, 0]

'И т о г о'

In [36]:
def parse_column_table(dataset, column_pos, end_index, column_name):
	col_data = []
	start_index_row = column_pos[0] + 1
	column_index = column_pos[1]
	statistics_col = {
		"str": 0,
		"int": 0
	}
	for i in range(start_index_row, end_index):
		row_data = dataset.iloc[i, column_index]
		cond_1 = (isinstance(row_data, str) or isinstance(row_data, int))
		cond_2 = column_name != row_data
		row = dataset.iloc[i]
		row_str = "".join(list(map(str, list(row.values))))
		cond_3 = not "и т о г о" in row_str.lower() 
		if cond_1 and cond_2 and cond_3:
			# print(row_data)
			col_data.append(row_data)
		
		if isinstance(row_data, str):
			statistics_col['str'] += 1
		elif isinstance(row_data, int):
			statistics_col['int'] += 1
	
	new_col_data = []
	for item in col_data:
		if statistics_col['str'] > statistics_col['int'] and isinstance(item, str):
				new_col_data.append(item)

		elif statistics_col['str'] < statistics_col['int'] and isinstance(item, int):
				new_col_data.append(item)
			
	return new_col_data


parse_column_table(
	dataset=dataset,
	column_pos=[200, 22],
	end_index=table_end_index_row,
	column_name='Всего семей'
)

[20,
 41,
 7,
 19,
 18,
 20,
 18,
 42,
 9,
 14,
 16,
 24,
 4,
 11,
 29,
 7,
 15,
 9,
 26,
 9,
 18,
 10,
 39,
 13,
 24,
 11,
 11,
 128,
 65,
 4,
 25,
 52,
 19,
 80,
 61,
 66,
 95,
 29]

In [37]:
columns_data

{'Наименование района': {'column_pos': [248, 11], 'data': []},
 'Всего семей': {'column_pos': [248, 25], 'data': []},
 'Всего детей': {'column_pos': [248, 53], 'data': []},
 'Семьи с 2-мя детьми-близнецами': {'column_pos': [], 'data': []},
 'Семьи с 3-мя детьми-близнецами': {'column_pos': [], 'data': []}}

In [38]:
dataset_object = {name: [] for name in columns_names}

for col_name in columns_names:
	col_info = columns_data[col_name]
	column_pos = col_info["column_pos"]
	# print(column_pos)
	col_data = parse_column_table(
		dataset=dataset,
		column_pos=column_pos,
		end_index=table_end_index_row,
		column_name=col_name
	)
	print(len(col_data))
	dataset_object[col_name] = col_data 

46
46
46


IndexError: list index out of range

In [31]:
new_dataset = pd.DataFrame(data=dataset_object)
new_dataset['Дата'] = "12/45/23"
new_dataset.to_csv(f"./{doc_name}_twins.csv", index=False)